In [1]:
#import necessary libraries
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import torch
from torchvision import transforms
from torch import nn, optim
import torch.utils.data as data_utils
from torch.optim import lr_scheduler
from torch.autograd import Variable
import matplotlib.pyplot as plt
%matplotlib inline

Reading the data

In [2]:
data = pd.read_csv('paf_8.csv')
data.head(3)

,Mean RR,Std RR,HF peak,LF power prc,HF power prc,SD1,SampEn,DFA ɑ1,Etiket PAF=1 Non_PAF=0,Kişi numarası
0,0.7109,0.1273,0.4438,0.0753,0.9335,0.1330,0.8583,0.2020,1,1
1,0.7052,0.1032,0.4064,0.0492,0.9257,0.1251,0.6426,0.1596,1,1
2,0.7022,0.1240,0.3823,0.1474,0.8228,0.1174,0.7113,0.2397,1,1


Print data shape and column info

In [3]:
print("data shape = ", data.shape, "\ndata column names = ", data.columns)

data shape =  (798, 10) 
data column names =  Index(['Mean RR', 'Std RR', 'HF peak', 'LF power prc', 'HF power prc', 'SD1',
       'SampEn', 'DFA ɑ1', 'Etiket\nPAF=1\nNon_PAF=0', 'Kişi numarası'],
      dtype='object')


Shuffle data by rows to have different input order to system

In [4]:
data = shuffle(data)
data.head()

,Mean RR,Std RR,HF peak,LF power prc,HF power prc,SD1,SampEn,DFA ɑ1,Etiket PAF=1 Non_PAF=0,Kişi numarası
666,1.0000,0.5864,0.2767,0.3347,0.7640,0.6616,0.7151,0.1840,0,82
676,0.5380,0.1183,0.0154,0.6212,0.3654,0.1113,0.5725,0.4185,0,83
156,0.9485,0.1558,0.2458,0.1701,0.7863,0.2144,0.7930,0.1215,1,27
56,0.5741,0.2085,0.4301,0.3840,0.5485,0.1821,0.3913,0.3656,1,10
658,0.8512,0.9117,0.3247,0.3248,0.6880,0.9222,0.5421,0.2900,0,81


Convert to torch Float Tensor

In [5]:
data = torch.FloatTensor(data.values)

Print sample data from new set

In [6]:
print("data[0,1] = ",data[0,1], "\ndata.shape =", data.shape)

data[0,1] =  tensor(0.5864) 
data.shape = torch.Size([798, 10])


In [7]:
len(data)

798

Columnwise normalizing the data

In [8]:
def normalize(t):
    t = t/t.max(dim=0)[0]
    t = (t-t.mean(dim=0))/t.std(dim=0)
    return t
t = torch.tensor([[1000, 3, 0.15], [500, 4, 0.35], [300, 8, 0.55]])
print(normalize(t))

tensor([[ 1.1094, -0.7559, -1.0000],
        [-0.2774, -0.3780,  0.0000],
        [-0.8321,  1.1339,  1.0000]])


Seperating data as input and output 8 column input and 1 column output (ignore 10th column)

In [9]:
data_in = data[:, 0:8]
data_out = data[:, 8:9]

Normalize only the input columns. Output is binary 0-1 categories. Combine input and output together

In [10]:
data_in = normalize(data_in)
data_in = data_in.float()
data = torch.cat((data_in, data_out), 1)

Sample data for view

In [11]:
x = data[:, 0:8]
x[0:3, :]

tensor([[ 2.6651,  2.6944, -0.4889, -0.6968,  1.3485,  3.5215,  1.4209, -1.3894],
        [ 0.1514, -0.3708, -1.4923,  0.4537, -0.1272, -0.2385,  0.6304, -0.1848],
        [ 2.3849, -0.1253, -0.6075, -1.3577,  1.4311,  0.4659,  1.8528, -1.7104]])

In [12]:
y = data[:, 8:9]
y[0:3, :]

tensor([[0.],
        [0.],
        [1.]])

Splitting the data for train (80%), validation (10%), test (10%)

In [13]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(data)*split_frac)
train_x, remaining_x = x[:split_idx], x[split_idx:]
train_y, remaining_y = y[:split_idx], y[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.size()), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape),
      "\nTrain label: \t\t{}".format(train_y.shape), 
      "\nValidation label: \t{}".format(val_y.shape),
      "\nTest label: \t\t{}".format(test_y.shape))

			Feature Shapes:
Train set: 		torch.Size([638, 8]) 
Validation set: 	torch.Size([80, 8]) 
Test set: 		torch.Size([80, 8]) 
Train label: 		torch.Size([638, 1]) 
Validation label: 	torch.Size([80, 1]) 
Test label: 		torch.Size([80, 1])


Preparing the train validation test dataloaders

In [14]:
train = data_utils.TensorDataset(train_x, train_y)
train_loader = data_utils.DataLoader(train, batch_size=8, shuffle=True)
validation = data_utils.TensorDataset(val_x, val_y)
validation_loader = data_utils.DataLoader(validation, batch_size=8, shuffle=True)
test = data_utils.TensorDataset(test_x, test_y)
test_loader = data_utils.DataLoader(test, batch_size=80, shuffle=True)

Network architecture

In [15]:
model = nn.Sequential(nn.Linear(8, 32),
                      nn.ReLU(),
                      nn.Dropout(0.1),
                      nn.Linear(32, 32),
                      nn.ReLU(),
                      nn.Dropout(0.1),
                      nn.Linear(32, 1),
                      nn.Sigmoid())

# Define the loss
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = lr_scheduler.StepLR(optimizer, step_size=250, gamma=0.5)
print(model)

Sequential(
  (0): Linear(in_features=8, out_features=32, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.1)
  (3): Linear(in_features=32, out_features=32, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.1)
  (6): Linear(in_features=32, out_features=1, bias=True)
  (7): Sigmoid()
)


Training and validation

In [16]:
epochs = 500

train_losses, val_losses = [], []
for e in range(epochs):
    running_loss = 0
    for inputs, labels in train_loader:
       
        optimizer.zero_grad()
        
        log_ps = model(inputs)
        
        
        loss = criterion(log_ps, labels)
        
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    else:
        
        val_loss = 0
        accuracy = 0
        model.eval()
        # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            for inputs, labels in validation_loader:

                log_ps = model(inputs)
                
                loss = criterion(log_ps, labels)                
                val_loss += loss.item()
                
                equals = log_ps == labels.view(*log_ps.shape)              
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        scheduler.step()        
        train_losses.append(running_loss/len(train_loader))
        val_losses.append(val_loss/len(validation_loader))
        if e % 10 == 9:
            print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Training Loss: {:.3f}.. ".format(running_loss/len(train_loader)),
                  "Validation Loss: {:.3f}.. ".format(val_loss/len(validation_loader)),
                  "Validation Accuracy: {:.3f}".format(accuracy/len(validation_loader)))
        model.train()

Epoch: 10/500..  Training Loss: 0.257..  Validation Loss: 0.279..  Validation Accuracy: 0.000
Epoch: 20/500..  Training Loss: 0.204..  Validation Loss: 0.241..  Validation Accuracy: 0.013
Epoch: 30/500..  Training Loss: 0.146..  Validation Loss: 0.236..  Validation Accuracy: 0.050
Epoch: 40/500..  Training Loss: 0.133..  Validation Loss: 0.308..  Validation Accuracy: 0.075
Epoch: 50/500..  Training Loss: 0.121..  Validation Loss: 0.251..  Validation Accuracy: 0.112
Epoch: 60/500..  Training Loss: 0.148..  Validation Loss: 0.419..  Validation Accuracy: 0.087
Epoch: 70/500..  Training Loss: 0.107..  Validation Loss: 0.488..  Validation Accuracy: 0.112
Epoch: 80/500..  Training Loss: 0.101..  Validation Loss: 0.408..  Validation Accuracy: 0.112
Epoch: 90/500..  Training Loss: 0.097..  Validation Loss: 0.302..  Validation Accuracy: 0.100
Epoch: 100/500..  Training Loss: 0.101..  Validation Loss: 0.318..  Validation Accuracy: 0.100
Epoch: 110/500..  Training Loss: 0.100..  Validation Loss: 

Testing the result

In [17]:
test_loss = 0
step = 0
accuracy = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        log_ps = model(inputs)
        
                
        loss = criterion(log_ps, labels)                
        test_loss += loss.item()
                
        equals = log_ps == labels.view(*log_ps.shape)              
        accuracy += torch.mean(equals.type(torch.FloatTensor))

    print("Test Loss: {:.3f}.. ".format(test_loss/len(test_loader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(test_loader)))

Test Loss: 0.906..  Test Accuracy: 0.262


Below is taken from internet for storing knowledge, not directly related. Don't consider that part

In [58]:
torch.save(model.state_dict(), 'checko.pth')

In [ ]:
state_dict = torch.load('checko.pth')
model.load_state_dict(state_dict)

In [ ]:
class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(8, 50)
        self.relu1 = nn.ReLU()
        self.dout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(50, 100)
        self.prelu = nn.PReLU(1)
        self.out = nn.Linear(100, 1)
        self.out_act = nn.Sigmoid()
        
    def forward(self, input_):
        a1 = self.fc1(input_)
        h1 = self.relu1(a1)
        dout = self.dout(h1)
        a2 = self.fc2(dout)
        h2 = self.prelu(a2)
        a3 = self.out(h2)
        y = self.out_act(a3)
        return y
    
net = Net()
opt = optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999))
criterion = nn.BCELoss()

In [ ]:
def train_epoch(net, opt, criterion, batch_size=50):
    model.train()
    losses = []
    for beg_i in range(0, x.size(0), batch_size):
        x_batch = x[beg_i:beg_i + batch_size, :]
        y_batch = y[beg_i:beg_i + batch_size, :]
        x_batch = Variable(x_batch)
        y_batch = Variable(y_batch)

        opt.zero_grad()
        # (1) Forward
        y_hat = net(x_batch)
        # (2) Compute diff
        loss = criterion(y_hat, y_batch)
        # (3) Compute gradients
        loss.backward()
        # (4) update weights
        opt.step()        
        losses.append(loss.data.numpy())
    return losses

In [ ]:
e_losses = []
num_epochs = 100
for e in range(num_epochs):
    e_losses += train_epoch(net, opt, criterion)
plt.plot(e_losses)